In [2]:
import logging
import os
import re
from datetime import datetime
from abc import ABC, abstractmethod

# Logger Class for Encapsulation
class Logger:
    def __init__(self, user_name):
        self.user_name = user_name
        self.log_file = self._setup_logger()
    
    def _setup_logger(self):
        os.makedirs("logs", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_filename = f"logs/{self.user_name}_session_{timestamp}.log"
        
        # Configure logger
        self.logger = logging.getLogger(self.user_name)
        self.logger.setLevel(logging.INFO)
        
        # Remove existing handlers and add new one
        if not self.logger.handlers:
            file_handler = logging.FileHandler(log_filename, 'w')
            formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
            file_handler.setFormatter(formatter)
            self.logger.addHandler(file_handler)
        
        self.logger.info("Logger initialized for user session.")
        return log_filename
    
    def get_logger(self):
        return self.logger

#(Abstraction)
class Validator(ABC):
    def __init__(self, logger):
        self.logger = logger

    @abstractmethod
    def validate(self, value):
        pass

#(Inheritance and Polymorphism)
class EmailValidator(Validator):
    def validate(self, value):
        pattern = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"
        if re.match(pattern, value):
            self.logger.info(f"Email validation success: {value}")
            return True, "VALID"
        else:
            self.logger.error(f"Email validation failed: {value}")
            return False, "INVALID: Email format is incorrect"

class AddressValidator(Validator):
    def validate(self, value):
        special_chars = ["!", "<", "@", "$", "%", "^", "*", "+", "=", "&", ">", "?"]
        if any(char in value for char in special_chars):
            self.logger.error(f"Address validation failed: {value}")
            return False, "INVALID: Address contains invalid special characters"
        self.logger.info(f"Address validation success: {value}")
        return True, "VALID"

class MobileNumberValidator(Validator):
    def validate(self, value):
        if value.isdigit() and len(value) == 10 and value[0] != "0":
            self.logger.info(f"Mobile number validation success: {value}")
            return True, "VALID"
        else:
            self.logger.error(f"Mobile number validation failed: {value}")
            return False, "INVALID: Mobile number must be 10 digits and not start with 0"

class ElectricityBillValidator(Validator):
    def validate(self, value):
        try:
            int(value)
            self.logger.info(f"Electricity bill validation success: {value}")
            return True, "VALID"
        except ValueError:
            self.logger.error(f"Electricity bill validation failed: {value}")
            return False, "INVALID: Electricity bill must be an integer"

class AreaValidator(Validator):
    def validate(self, value):
        try:
            float(value)
            self.logger.info(f"Area validation success: {value}")
            return True, f"VALID: {value} sq mt"
        except ValueError:
            self.logger.error(f"Area validation failed: {value}")
            return False, "INVALID: Area must be a floating-point number"

# (Encapsulation- managing user and validators)
class Application:
    def __init__(self):
        self.user_name = input("Enter your name: ").strip()
        self.logger = Logger(self.user_name).get_logger()
        self.validators = {
            "email": EmailValidator(self.logger),
            "address": AddressValidator(self.logger),
            "mobile": MobileNumberValidator(self.logger),
            "bill": ElectricityBillValidator(self.logger),
            "area": AreaValidator(self.logger),
        }

    def run(self):
        inputs = {
            "email": "Enter email ID: ",
            "address": "Enter address: ",
            "mobile": "Enter mobile number: ",
            "bill": "Enter monthly electricity bill: ",
            "area": "Enter area in square meters: ",
        }
        for key, prompt in inputs.items():
            value = input(prompt).strip()
            valid, message = self.validators[key].validate(value)
            print(message)
        
        self.logger.info("Session ended.")
        print(f"Session log saved to {Logger(self.user_name).log_file}")

# Ensure single execution
if __name__ == "__main__":
    app = Application()
    app.run()


VALID
VALID
INVALID: Mobile number must be 10 digits and not start with 0
VALID
VALID: 500 sq mt
Session log saved to logs/Kalashri_session_20250121_152043.log
